In [ ]:
"""
Módulo para entrenamiento y análisis de Redes Neuronales Bayesianas (BNN).

Este módulo proporciona una interfaz para entrenar y analizar modelos BNN
para predecir niveles de NO2, capturando tanto la predicción como la incertidumbre.
"""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from datetime import timedelta
import seaborn as sns
import joblib
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from typing import Dict, List, Tuple, Optional
import warnings
import statsmodels.api as sm
from scipy.stats import zscore
import math
import argparse

warnings.filterwarnings('ignore')
torch.classes.__path__ = [] # add this line to manually set it to empty.

# Constante para estabilidad numérica
EPS = 1e-6

# ==================== COMPONENTES BNN (Basado en bayesian_example.py) ====================

class MFVILinear(nn.Module):
    """Capa lineal Bayesiana usando Inferencia Variacional de Campo Medio (MFVI)."""

    def __init__(self, dim_in, dim_out, prior_weight_std=1.0, prior_bias_std=1.0, init_std=0.05, device=None, dtype=None):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out

        self.weight_mean = nn.Parameter(torch.empty((dim_out, dim_in), **factory_kwargs))
        self.bias_mean = nn.Parameter(torch.empty(dim_out, **factory_kwargs))
        self._weight_std_param = nn.Parameter(torch.empty((dim_out, dim_in), **factory_kwargs))
        self._bias_std_param = nn.Parameter(torch.empty(dim_out, **factory_kwargs))
        
        self.reset_parameters(init_std)

        prior_mean = 0.0
        self.register_buffer('prior_weight_mean', torch.full_like(self.weight_mean, prior_mean))
        self.register_buffer('prior_weight_std', torch.full_like(self._weight_std_param, prior_weight_std))
        self.register_buffer('prior_bias_mean', torch.full_like(self.bias_mean, prior_mean))
        self.register_buffer('prior_bias_std', torch.full_like(self._bias_std_param, prior_bias_std))

    def reset_parameters(self, init_std=0.05):
        nn.init.kaiming_uniform_(self.weight_mean, a=math.sqrt(5))
        bound = self.dim_in ** -0.5 if self.dim_in > 0 else 0
        nn.init.uniform_(self.bias_mean, -bound, bound)
        _init_std_param = np.log(init_std)
        self._weight_std_param.data = torch.full_like(self.weight_mean, _init_std_param)
        self._bias_std_param.data = torch.full_like(self.bias_mean, _init_std_param)

    @property
    def weight_std(self):
        return torch.clamp(torch.exp(self._weight_std_param), min=EPS)

    @property
    def bias_std(self):
        return torch.clamp(torch.exp(self._bias_std_param), min=EPS)

    def kl_divergence(self):
        q_weight = dist.Normal(self.weight_mean, self.weight_std)
        p_weight = dist.Normal(self.prior_weight_mean, self.prior_weight_std)
        kl = dist.kl_divergence(q_weight, p_weight).sum()
        
        q_bias = dist.Normal(self.bias_mean, self.bias_std)
        p_bias = dist.Normal(self.prior_bias_mean, self.prior_bias_std)
        kl += dist.kl_divergence(q_bias, p_bias).sum()
        return kl

    def forward(self, input):
        weight = self._normal_sample(self.weight_mean, self.weight_std)
        bias = self._normal_sample(self.bias_mean, self.bias_std)
        return F.linear(input, weight, bias)

    def _normal_sample(self, mean, std):
        epsilon = torch.randn_like(std)
        return mean + std * epsilon

def make_mfvi_bnn(layer_sizes, activation='GELU', **layer_kwargs):
    nonlinearity = getattr(nn, activation)() if isinstance(activation, str) else activation
    net = nn.Sequential()
    for i, (dim_in, dim_out) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        net.add_module(f'MFVILinear{i}', MFVILinear(dim_in, dim_out, **layer_kwargs))
        if i < len(layer_sizes) - 2:
            net.add_module(f'Nonlinarity{i}', nonlinearity)
    return net

def kl_divergence_model(bnn):
    kl = 0.0
    for module in bnn.modules():
        if hasattr(module, 'kl_divergence'):
            kl += module.kl_divergence()
    return kl

def gauss_loglik(y, y_pred, log_noise_var):
    l2_dist = (y - y_pred).pow(2).sum(-1)
    return -0.5 * (log_noise_var + math.log(2 * math.pi) + l2_dist * torch.exp(-log_noise_var))

def test_nll(y, y_pred, log_noise_var):
    nll_samples = -gauss_loglik(y, y_pred, log_noise_var) # Shape (K, N_test)
    nll = -torch.logsumexp(-nll_samples, dim=0) + math.log(nll_samples.shape[0])
    return nll.mean()

# ==================== CLASE DATASET PYTORCH ====================

class BayesianDataset(Dataset):
    def __init__(self, features, target):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.target = torch.tensor(target.values, dtype=torch.float32).reshape(-1, 1)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.target[idx]

# ==================== CLASE BNN TRAINER ====================

class BNNTrainer:
    """Clase para encapsular el entrenamiento y análisis de BNNs."""
    
    def __init__(self):
        self.df_master = None
        self.model = None
        self.scaler_dict = {}
        self.scaler_target = None
        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        print(f"Using device: {self.device}")

    def load_data(self) -> pd.DataFrame:
        try:
            df = pd.read_parquet('../data/super_processed/7_4_no2_with_traffic_and_1meteo_and_1trafic_id.parquet')
            df['fecha'] = pd.to_datetime(df['fecha'])
            return df
        except Exception as e:
            print(f"Error al cargar los datos: {str(e)}")
            raise e
    
    def create_cyclical_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df['day_of_week'] = df['fecha'].dt.dayofweek
        df['day_of_year'] = df['fecha'].dt.dayofyear
        df['month'] = df['fecha'].dt.month
        df['hour'] = df['fecha'].dt.hour
        df['day'] = df['fecha'].dt.day
        df['weekend'] = (df['day_of_week'] >= 5).astype(int)
        df['season'] = df['month'].apply(lambda x: (x%12 + 3)//3) # 1:winter, 2:spring, 3:summer, 4:autumn
        
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        return df

    def remove_outliers(self, df: pd.DataFrame, method: str, columns: List[str]) -> pd.DataFrame:
        if method == 'none': return df
        df_filtered = df.copy()
        for col in columns:
            if col not in df_filtered.columns: continue
            if method == 'iqr':
                Q1, Q3 = df_filtered[col].quantile(0.25), df_filtered[col].quantile(0.75)
                IQR = Q3 - Q1
                if IQR > 0:
                    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
                    df_filtered = df_filtered[(df_filtered[col] >= lower) & (df_filtered[col] <= upper)]
            elif method == 'zscore':
                df_filtered = df_filtered[np.abs(zscore(df_filtered[col], nan_policy='omit')) < 3]
        return df_filtered

    def split_data(self, df: pd.DataFrame, split_date: pd.Timestamp) -> Tuple[pd.DataFrame, pd.DataFrame]:
        train = df[df['fecha'] < split_date].copy()
        test = df[df['fecha'] >= split_date].copy()
        return train, test

    def scale_features(self, X_train: pd.DataFrame, X_test: pd.DataFrame, features: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame, Dict]:
        scaler_dict = {}
        X_train_s, X_test_s = X_train.copy(), X_test.copy()
        for feature in features:
            if feature in X_train.columns and pd.api.types.is_numeric_dtype(X_train[feature]):
                scaler = StandardScaler()
                X_train_s[feature] = scaler.fit_transform(X_train[[feature]]).flatten()
                X_test_s[feature] = scaler.transform(X_test[[feature]]).flatten()
                scaler_dict[feature] = scaler
        return X_train_s, X_test_s, scaler_dict

    def scale_target(self, y_train: pd.Series) -> Tuple[pd.Series, StandardScaler]:
        scaler = StandardScaler()
        y_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
        return pd.Series(y_scaled, index=y_train.index, name=y_train.name), scaler

    def train_bnn_model(self, X_train, y_train, train_config):
        # Setup
        dataset = BayesianDataset(X_train, y_train)
        dataloader = DataLoader(dataset, batch_size=train_config['batch_size'], shuffle=True)
        
        layer_sizes = [X_train.shape[1]] + train_config['hidden_dims'] + [1]
        model = make_mfvi_bnn(layer_sizes, activation=train_config['activation'], device=self.device).to(self.device)
        log_noise_var = nn.Parameter(torch.ones(1, device=self.device) * -3.0)
        
        params = list(model.parameters()) + [log_noise_var]
        optimizer = torch.optim.Adam(params, lr=train_config['learning_rate'])
        
        # Training Loop
        logs = []
        N_data = len(dataset)
        
        print("Iniciando entrenamiento...")
        for epoch in range(train_config['n_epochs']):
            model.train()
            epoch_nll, epoch_kl = 0, 0
            for x_batch, y_batch in dataloader:
                x_batch, y_batch = x_batch.to(self.device), y_batch.to(self.device)
                optimizer.zero_grad()
                
                y_pred = model(x_batch)
                nll = -gauss_loglik(y_batch, y_pred, log_noise_var).mean()
                kl = kl_divergence_model(model)
                loss = nll + train_config['beta'] * kl / N_data
                
                loss.backward()
                optimizer.step()
                
                epoch_nll += nll.item() * len(x_batch)
                epoch_kl += kl.item()
            
            logs.append({'nll': epoch_nll / N_data, 'kl': epoch_kl / N_data})
            if (epoch + 1) % 50 == 0:
                 print(f"Epoch {epoch+1}/{train_config['n_epochs']} | NLL: {logs[-1]['nll']:.3f} | KL: {logs[-1]['kl']:.3f}")
        
        print("Entrenamiento completado.")
        return model, log_noise_var, logs

    def predict(self, model, X_test, K, log_noise_var):
        model.eval()
        X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(self.device)
        
        with torch.no_grad():
            y_preds = torch.stack([model(X_test_tensor) for _ in range(K)], dim=0)
        
        pred_mean = y_preds.mean(0)
        epistemic_uncertainty = y_preds.var(0).sqrt()
        aleatoric_uncertainty = torch.exp(0.5 * log_noise_var).expand_as(pred_mean)
        total_uncertainty = (epistemic_uncertainty**2 + aleatoric_uncertainty**2).sqrt()
        
        return {
            'y_preds_all': y_preds,
            'mean': pred_mean,
            'epistemic_std': epistemic_uncertainty,
            'aleatoric_std': aleatoric_uncertainty,
            'total_std': total_uncertainty
        }

    def evaluate_model(self, predictions, y_test, y_test_scaled, scaler_target, log_noise_var):
        # Unscale predictions
        pred_mean_scaled = predictions['mean'].detach().cpu().numpy()
        pred_mean = scaler_target.inverse_transform(pred_mean_scaled).flatten()
        
        total_std_scaled = predictions['total_std'].detach().cpu().numpy().flatten()
        unscaled_std = total_std_scaled * scaler_target.scale_[0]
        
        epistemic_std = predictions['epistemic_std'].detach().cpu().numpy().flatten()
        epistemic_unescaled_std = epistemic_std * scaler_target.scale_[0]

        # Create DataFrame
        df_preds = pd.DataFrame({
            'prediction': pred_mean,
            'epistemic_uncertainty': epistemic_unescaled_std
        })

        # Save to Excel
        df_preds.to_csv('bnn_predictions_with_epistemic_uncertainty.csv')


        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, pred_mean))
        r2 = r2_score(y_test, pred_mean)
        mae = mean_absolute_error(y_test, pred_mean)
        
        # Test NLL
        y_test_tensor = torch.tensor(y_test_scaled.values, dtype=torch.float32).reshape(-1, 1).to(self.device)
        nll = test_nll(y_test_tensor, predictions['y_preds_all'], log_noise_var).item()
        
        return {
            'rmse': rmse, 'r2': r2, 'mae': mae, 'test_nll': nll,
            'y_pred': pred_mean, 'y_pred_std': unscaled_std
        }
    
    def save_model(self, path, model, log_noise_var, scaler_dict, scaler_target, feature_names):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        model_state = {
            'model_state_dict': model.state_dict(),
            'log_noise_var': log_noise_var,
            'scaler_dict': scaler_dict,
            'scaler_target': scaler_target,
            'feature_names': feature_names
        }
        # Guardar en formato pickle
        joblib.dump(model_state, path)

    def load_model(self, path, layer_sizes, activation):
        if not os.path.exists(path):
            print(f"Modelo no encontrado en: {path}")
            return None
        
        # Cargar desde formato pickle
        model_state = joblib.load(path)
        model = make_mfvi_bnn(layer_sizes, activation=activation, device=self.device).to(self.device)
        model.load_state_dict(model_state['model_state_dict'])
        log_noise_var = model_state['log_noise_var'].to(self.device)
        
        return model, log_noise_var, model_state['scaler_dict'], model_state['scaler_target'], model_state['feature_names']

# ==================== FUNCIONES DE VISUALIZACIÓN Y REPORTE ====================

def print_model_metrics(metrics: Dict):
    print("\n📊 Métricas de Evaluación")
    print(f"  - RMSE: {metrics['rmse']:.2f} µg/m³ (Error Cuadrático Medio)")
    print(f"  - R² Score: {metrics['r2']:.3f} (Coeficiente de Determinación)")
    print(f"  - MAE: {metrics['mae']:.2f} µg/m³ (Error Absoluto Medio)")
    print(f"  - Test NLL: {metrics['test_nll']:.3f} (Log-verosimilitud Negativa en Test, menor es mejor)")

def save_temporal_predictions_plot(test_df: pd.DataFrame, y_pred: np.ndarray, y_pred_std: np.ndarray, filename: str):
    print(f"\n📈 Guardando gráfico de predicciones en {filename}")
    df_plot = test_df[['fecha', 'no2_value']].copy()
    df_plot['Predicción'] = y_pred
    df_plot['Incertidumbre_std'] = y_pred_std
    df_plot = df_plot.set_index('fecha')

    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(df_plot.index, df_plot['no2_value'], label='Valor Real', alpha=0.9)
    ax.plot(df_plot.index, df_plot['Predicción'], label='Predicción Media (BNN)', linestyle='--')
    
    ax.fill_between(df_plot.index, 
                    df_plot['Predicción'] - 2 * df_plot['Incertidumbre_std'],
                    df_plot['Predicción'] + 2 * df_plot['Incertidumbre_std'],
                    color='orange', alpha=0.3, label='Incertidumbre Total (±2 std)')

    ax.set_title("Predicciones del Modelo BNN vs. Datos Reales")
    ax.set_ylabel("Concentración NO₂ (µg/m³)")
    ax.legend()
    fig.autofmt_xdate()
    plt.savefig(filename)
    plt.close(fig)

def save_training_loss_plot(logs: List[Dict], filename: str):
    print(f"📉 Guardando curvas de entrenamiento en {filename}")
    log_df = pd.DataFrame(logs)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.plot(log_df['nll'])
    ax1.set_title("Negative Log-Likelihood (NLL)")
    ax1.set_xlabel("Época")
    ax2.plot(log_df['kl'])
    ax2.set_title("KL Divergence")
    ax2.set_xlabel("Época")
    plt.savefig(filename)
    plt.close(fig)

# ==================== FUNCIÓN PRINCIPAL ====================

def main(config):
    print("🧠 Entrenamiento de Red Neuronal Bayesiana (BNN)")
    trainer = BNNTrainer()
    
    print("\nCargando datos...")
    trainer.df_master = trainer.load_data()
    if trainer.df_master.empty: return

    print(f"\n⚙️ Configuración del Experimento:")
    print(f"  - Sensor: {config['sensor_id']}")
    print(f"  - Fecha de división: {config['split_date']}")
    print(f"  - Filtrado de Outliers: {config['outlier_method']}")
    
    df_sensor = trainer.df_master[trainer.df_master['id_no2'] == config['sensor_id']]
    if df_sensor.empty:
        print(f"Error: No se encontraron datos para el sensor {config['sensor_id']}")
        return

    print("\nProcesando datos...")
    df_processed = trainer.convert_units(df_sensor.copy())
    df_processed = trainer.create_cyclical_features(df_processed)
    
    train_df, test_df = trainer.split_data(df_processed, pd.to_datetime(config['split_date']))
    
    train_df = trainer.remove_outliers(train_df, config['outlier_method'], ['no2_value'])
    
    all_features = [c for c in df_sensor.columns if c not in ['fecha', 'id_no2', 'no2_value'] and pd.api.types.is_numeric_dtype(df_sensor[c])]
    temp_df = trainer.create_cyclical_features(df_sensor[['fecha']].copy())
    all_features.extend([c for c in temp_df.columns if c not in df_sensor.columns])
    
    if config['features'] == ['all']:
        selected_features = all_features
    else:
        selected_features = [f for f in config['features'] if f in all_features]
    
    print(f"\nUsando {len(selected_features)} caracteristicas: {selected_features}")

    X_train, X_test = train_df[selected_features], test_df[selected_features]
    y_train, y_test = train_df['no2_value'], test_df['no2_value']
    
    X_train, X_test, scaler_dict = trainer.scale_features(X_train, X_test, selected_features)
    y_train_scaled, scaler_target = trainer.scale_target(y_train)
    y_test_scaled, _ = trainer.scale_target(y_test) # Scaler is not needed here
    
    print(f"Datos de entrenamiento: {len(X_train)}, Datos de prueba: {len(X_test)}")

    model, log_noise_var, logs = trainer.train_bnn_model(X_train, y_train_scaled, config['train_config'])
    
    print("\nEvaluando modelo...")
    predictions = trainer.predict(model, X_test, config['K_predict'], log_noise_var)
    metrics = trainer.evaluate_model(predictions, y_test, y_test_scaled, scaler_target, log_noise_var)

    print("\n🔬 Resultados del Análisis")
    print_model_metrics(metrics)
    
    # Crear carpeta de resultados si no existe
    output_dir = f"../data/models/"
    
    #save_temporal_predictions_plot(test_df, metrics['y_pred'], metrics['y_pred_std'], filename=os.path.join(output_dir, "predictions.png"))
    #save_training_loss_plot(logs, filename=os.path.join(output_dir, "training_loss.png"))
    
    model_path = os.path.join(output_dir, f"bnn_model_{config['sensor_id']}.pkl")
    print(f"\n💾 Guardando modelo en {model_path}")
    trainer.save_model(model_path, model, log_noise_var, scaler_dict, scaler_target, selected_features)
    print("Proceso completado.")


In [8]:
parser = argparse.ArgumentParser(description="Entrenar un modelo BNN para predicción de NO2.")
parser.add_argument("--sensor_id", type=str, default="28079008", help="ID del sensor de NO2 a utilizar.")
parser.add_argument("--split_date", type=str, default="2024-01-01", help="Fecha para dividir datos en entrenamiento/prueba (YYYY-MM-DD).")
parser.add_argument("--outlier_method", type=str, default="iqr", choices=['none', 'iqr', 'zscore'], help="Método de filtrado de outliers.")
parser.add_argument("--hidden_dims", type=str, default="50,50", help="Dimensiones de capas ocultas, separadas por comas.")
parser.add_argument("--activation", type=str, default="GELU", choices=['GELU', 'ReLU', 'LeakyReLU'], help="Función de activación.")
parser.add_argument("--lr", type=float, default=0.01, help="Tasa de aprendizaje.")
parser.add_argument("--epochs", type=int, default=25, help="Número de épocas de entrenamiento.")
parser.add_argument("--batch_size", type=int, default=256, help="Tamaño del batch.")
parser.add_argument("--beta", type=float, default=1.0, help="Peso para la divergencia KL (beta).")
parser.add_argument("--k_predict", type=int, default=100, help="Número de muestras Monte Carlo para predicción.")

args, unknown = parser.parse_known_args()

config = {
    'sensor_id': args.sensor_id,
    'split_date': args.split_date,
    'outlier_method': args.outlier_method,
    'features': ['all'], # se usan todas por defecto
    'K_predict': args.k_predict,
    'train_config': {
        'learning_rate': args.lr,
        'n_epochs': args.epochs,
        'batch_size': args.batch_size,
        'hidden_dims': [int(d.strip()) for d in args.hidden_dims.split(',')],
        'activation': args.activation,
        'beta': args.beta,
    }
}

main(config) 

🧠 Entrenamiento de Red Neuronal Bayesiana (BNN)
Using device: cpu

Cargando datos...

⚙️ Configuración del Experimento:
  - Sensor: 28079008
  - Fecha de división: 2024-01-01
  - Filtrado de Outliers: iqr

Procesando datos...

Usando 31 caracteristicas: ['longitud_no2', 'latitud_no2', 'distance_m', 'intensidad', 'carga', 'ocupacion', 'vmed', 'latitud_meteo', 'longitud_meteo', 'd2m', 't2m', 'ssr', 'ssrd', 'u10', 'v10', 'sp', 'tp', 'day_of_week', 'day_of_year', 'month', 'year', 'weekend', 'season', 'hour', 'day', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos']
Datos de entrenamiento: 48794, Datos de prueba: 7777
Iniciando entrenamiento...
Entrenamiento completado.

Evaluando modelo...

🔬 Resultados del Análisis

📊 Métricas de Evaluación
  - RMSE: 13.00 µg/m³ (Error Cuadrático Medio)
  - R² Score: 0.539 (Coeficiente de Determinación)
  - MAE: 9.22 µg/m³ (Error Absoluto Medio)
  - Test NLL: 1.383 (Log-verosimilitud Negativa en Test, menor es mejor)

